# LeRobot EnvHub + LIBERO (Fold Clothes)

Installs all LeRobot env extras and shows how to load a LIBERO fold-clothes task and render a short random rollout.


In [ ]:
# Install LeRobot with only the LIBERO extra (folding tasks)
!pip install "lerobot[libero]==0.4.1" numpy==1.26.0 #matplotlib


  Using cached numpy-1.26.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (58 kB)
  Using cached hf_libero-0.1.3.tar.gz (3.0 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached hf_egl_probe-1.0.2.tar.gz (217 kB)
  Preparing metadata (setup.py) ... done
  Using cached hydra_core-1.3.2-py3-none-any.whl.metadata (5.5 kB)
  Using cached robomimic-0.2.0.tar.gz (192 kB)
  Preparing metadata (setup.py) ... done
  Using cached robosuite-1.4.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached bddl-1.0.1.tar.gz (164 kB)
  Preparing metadata (setup.py) ... done
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl.metadata (2.7 kB)
  Using cached mujoco-3.3.7-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (41 kB)
  Using cached tensorboardx-2.6.4-py3-none-any.whl.metadata (6.2 kB)
  Using cached egl_probe-1.0.2.tar.gz (217 kB)
  Preparing metadata (

## Find a LIBERO fold-clothes task

Lists tasks in the LIBERO suite and picks the first one containing the word 'fold'. Only the LIBERO extra is installed to keep dependencies minimal.


In [ ]:
from libero.libero import benchmark

suite_name = "libero_90"  # richer set of tasks (includes folding)
tasks = benchmark.get_benchmark_dict()[suite_name]().tasks
fold_ids = [i for i, t in enumerate(tasks) if 'fold' in t.name.lower()]

print(f"Suite {suite_name} has {len(tasks)} tasks")
print("Fold-related task ids:", fold_ids)
for i in fold_ids[:5]:
    print(f"{i:02d}: {tasks[i].name}")

target_task_id = fold_ids[0] if fold_ids else 0
print(f"Using task_id={target_task_id}")


## Create the LIBERO env for the fold-clothes task
Uses `create_libero_envs` to build a vectorized env filtered to the chosen task id.


In [ ]:
import gymnasium as gym
from lerobot.envs.libero import create_libero_envs

env_map = create_libero_envs(
    task=suite_name,
    n_envs=1,
    gym_kwargs={
        'task_ids': [target_task_id],
        'obs_type': 'pixels',
        'render_mode': 'rgb_array',
        'observation_height': 256,
        'observation_width': 256,
        'visualization_width': 256,
        'visualization_height': 256,
    },
    env_cls=gym.vector.SyncVectorEnv,
)
env = env_map[suite_name][target_task_id]

obs, info = env.reset(seed=0)
pixel_keys = list(obs['pixels'].keys())
print("Pixel camera keys:", pixel_keys)
print("Frame shape:", obs['pixels'][pixel_keys[0]].shape)
print("Action space:", env.action_space)


## Run a short random rollout and collect frames


In [ ]:
frames = []
obs, info = env.reset(seed=0)
for step in range(50):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    frame = obs['pixels'][pixel_keys[0]][0] if obs['pixels'][pixel_keys[0]].ndim == 4 else obs['pixels'][pixel_keys[0]]
    frames.append(frame)
    if terminated[0] or truncated[0]:
        break

env.close()
print(f"Collected {len(frames)} frames; last reward={float(reward[0]) if hasattr(reward, '__len__') else float(reward):.3f}")


## Visualize the rollout


In [ ]:
import matplotlib.pyplot as plt
from IPython.display import HTML
import matplotlib.animation as animation

fig, ax = plt.subplots(figsize=(4, 4))
ax.axis('off')
img = ax.imshow(frames[0])

def animate(i):
    img.set_data(frames[i])
    return [img]

ani = animation.FuncAnimation(fig, animate, frames=len(frames), interval=50)
HTML(ani.to_jshtml())
